<a href="https://colab.research.google.com/github/teamgaon/KLUE/blob/main/220228_sm_FINAL_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리, 패키지

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers

     |████████████████████████████████| 311 kB 5.2 MB/s 
     |████████████████████████████████| 3.5 MB 78.9 MB/s 
     |████████████████████████████████| 134 kB 98.4 MB/s 
     |████████████████████████████████| 1.1 MB 79.4 MB/s 
     |████████████████████████████████| 212 kB 88.0 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 144 kB 90.9 MB/s 
     |████████████████████████████████| 271 kB 70.7 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 596 kB 85.8 MB/s 
     |████████████████████████████████| 895 kB 90.6 MB/s 
     |████████████████████████████████| 6.8 MB 77.1 MB/s 
     |████████████████████████████████| 1.2 MB 78.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 58 kB 3.0 MB/s 
     |████████████████████████████████| 149.9 

In [3]:
from accelerate import Accelerator

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from transformers import TrainingArguments, Trainer
from transformers import AutoModel,AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import gc
from transformers import AdamW
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup
from tqdm.auto import tqdm
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')
from accelerate import notebook_launcher

# 데이터

In [5]:
PATH =  '/content/drive/MyDrive/KLUE'

# 데이콘 데이터셋
train = pd.read_csv(os.path.join(PATH, 'train_data.csv'), encoding='utf-8')
test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')

# 카카오 데이터셋
kakao_snli = pd.read_csv('/content/drive/MyDrive/KLUE/snli_1.0_train.ko.tsv', sep='\t', encoding='utf-8')
kakao_snli = kakao_snli[:100000]
kakao_dev = pd.read_csv('/content/drive/MyDrive/KLUE/xnli.dev.ko.tsv', sep='\t', encoding='utf-8')
kakao_test = pd.read_csv('/content/drive/MyDrive/KLUE/xnli.test.ko.tsv', sep='\t', encoding='utf-8')
kakao_dev = pd.concat([kakao_dev,kakao_test, kakao_snli])
kakao_dev.rename(columns = {'sentence1':'premise','sentence2':'hypothesis','gold_label':'label'},inplace=True)

# KLUE 데이터셋
klue_dev = pd.read_json('/content/drive/MyDrive/KLUE/klue-nli-v1.1_dev.json')
klue_dev = klue_dev[['premise', 'hypothesis', 'gold_label']]
klue_dev.rename(columns = {'gold_label':'label'}, inplace=True)

# 데이콘, 카카오, KLUE 데이터셋 병합
train = pd.concat([train, klue_dev, kakao_dev], axis=0)
train = train.reset_index(drop=True)
train['index'] = train.index
train = train.dropna()
train = train.reset_index(drop=True)
train['index'] = train.index
train

,index,premise,hypothesis,label
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral
...,...,...,...,...
134514,134514,같은 방향으로 보이는 숲이 우거진 지역에 의상을 입은 사람들이 모여 있다.,사람들은 의상을 입는다,entailment
134515,134515,검은 원피스에 커다란 흰 활을 든 소녀가 카메라를 등지고 서 있다.,하이힐을 신은 남자,contradiction
134516,134516,검은 원피스에 커다란 흰 활을 든 소녀가 카메라를 등지고 서 있다.,서 있는 소녀,entailment
134517,134517,검은 원피스에 커다란 흰 활을 든 소녀가 카메라를 등지고 서 있다.,사진 촬영 준비를 하고 있는 소녀,neutral


# 토크나이저, 함수

In [6]:
from transformers import AutoTokenizer

checkpoint = 'klue/roberta-large'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [7]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

In [8]:
def label_to_num(label):
    label_dict = {"entailment": 0, "contradiction": 1, "neutral": 2, "answer": 3}
    num_label = []

    for v in label:
        num_label.append(label_dict[v])
    
    return num_label

In [9]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 학습

In [10]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=16, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=16, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 3
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 10
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )

  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/220228/model' + str(fold), save_function=accelerator.save)

In [11]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)

print('--------------------------------')

tokenized_train = tokenizer(
  list(train['premise']),
  list(train['hypothesis']),
  return_tensors="pt",
  max_length=128, # Max_Length = 190
  padding=True,
  truncation=True,
  add_special_tokens=True
  )
  
for fold, (train_ids, test_ids) in enumerate(kfold.split(train, train['label'])):
  print(f'FOLD {fold}')

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

  train_label = label_to_num(train['label'].values)

  train_dataset = BERTDataset(tokenized_train, train_label)

  notebook_launcher(training_function)

--------------------------------
FOLD 0
Launching a training on 8 TPU cores.


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'cl

  0%|          | 0/8410 [00:00<?, ?it/s]

  0%|          | 0/8410 [00:00<?, ?it/s]

  0%|          | 0/8410 [00:00<?, ?it/s]

...

  0%|          | 0/8410 [00:00<?, ?it/s]

  0%|          | 0/8410 [00:00<?, ?it/s]

  0%|          | 0/8410 [00:00<?, ?it/s]

  0%|          | 0/8410 [00:00<?, ?it/s]

epoch 1 train acc 0.7878269916765755
epoch 1 train acc 0.7874554102259215
epoch 1 train acc 0.7875297265160524
epoch 1 train acc 0.7868608799048752
epoch 1 train acc 0.790205112960761
epoch 1 train acc 0.7793549346016647
epoch 1 train acc 0.7820303210463734
epoch 1 train acc 0.7855975029726516
epoch 1 test acc 0.8566350710900474
epoch 1 test acc 0.8382701421800948
epoch 1 test acc 0.8477488151658767
epoch 1 test acc 0.8504146919431279
epoch 1 test acc 0.8486374407582938
epoch 1 test acc 0.8489336492890995
epoch 1 test acc 0.8421208530805687
epoch 1 test acc 0.8495260663507109
epoch 2 train acc 0.8744797859690844
epoch 2 train acc 0.8741082045184304
epoch 2 train acc 0.875371581450654
epoch 2 train acc 0.8703923900118906
epoch 2 train acc 0.8752229488703924
epoch 2 train acc 0.8689803804994055
epoch 2 train acc 0.8700951248513674
epoch 2 train acc 0.8723989298454221
epoch 2 test acc 0.8643364928909952
epoch 2 test acc 0.8631516587677726
epoch 2 test acc 0.8584123222748815
epoch 2 test a

# 추론

In [12]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

test_label = label_to_num(test['label'].values)

tokenized_test = tokenizer(
    list(test['premise']),
    list(test['hypothesis']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

for fold in range(5):

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 3
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/220228/model' + str(fold), num_labels=3)
  model.resize_token_embeddings(tokenizer.vocab_size)
  accelerator = Accelerator()
  model = accelerator.unwrap_model(model)

  output_pred = []
  output_prob = []

  model, dataloader= accelerator.prepare(model, dataloader)

  model.eval()

  for i, data in enumerate(tqdm(dataloader)):
      with torch.no_grad():
          outputs = model(
              input_ids=data['input_ids'],
              attention_mask=data['attention_mask']
          )
      logits = outputs[0]
      prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      result = np.argmax(logits, axis=-1)
      output_pred.append(result)
      output_prob.append(prob)
      
  pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

  def num_to_label(label):
      label_dict = {0: "entailment", 1: "contradiction", 2: "neutral"}
      str_label = []

      for i, v in enumerate(label):
          str_label.append([i,label_dict[v]])
      
      return str_label

  answer = num_to_label(pred_answer)

  df_label = pd.DataFrame(answer, columns=['index', 'label'])
  df_prob = pd.DataFrame(output_prob)

  df_label.to_csv('/content/drive/MyDrive/220228/pred_label'+str(fold)+'.csv', index=False)
  df_prob.to_csv('/content/drive/MyDrive/220228/pred_prob'+str(fold)+'.csv', index=False)

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

## softvoting

In [405]:
pred0 = pd.read_csv('/content/drive/MyDrive/sm_220228/pred_prob0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/sm_220228/pred_prob1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/sm_220228/pred_prob2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/sm_220228/pred_prob3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/sm_220228/pred_prob4.csv')

In [406]:
pred = pd.DataFrame((np.array(pred0) + np.array(pred1) + np.array(pred2) + np.array(pred3) + np.array(pred4))/5)

In [407]:
test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')

In [408]:
test = pd.concat([test, pred], axis=1)

In [409]:
def num_to_label(label):
    label_dict = {0: "entailment", 1: "contradiction", 2: "neutral"}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label(np.argmax(np.array(pred), axis=-1))

In [410]:
test['label'] = pd.DataFrame(answer)[1]

In [411]:
for i, sent in enumerate(test['premise'].unique()):
  if len(test[test['premise'] == sent]) == 3:
    continue
  else :
    print(sent)

한겨례평화통일포럼은 18일 안산시 드림스타트 아동들에게 전달해 달라며 쌀 48포를 기탁했다.


In [412]:
test[test['premise'] == '한겨례평화통일포럼은 18일 안산시 드림스타트 아동들에게 전달해 달라며 쌀 48포를 기탁했다.']

,index,premise,hypothesis,label,0,1,2
25,25,한겨례평화통일포럼은 18일 안산시 드림스타트 아동들에게 전달해 달라며 쌀 48포를 ...,한겨례평화통일포럼은 20키로 쌀로 기탁했다.,contradiction,0.010205,0.977261,0.012534


In [415]:
def find_max(test):
  return max(test[0], test[1], test[2])

test['max'] = test.apply(find_max, axis=1)

In [416]:
test

,index,premise,hypothesis,label,0,1,2,max
0,0,다만 조금 좁아서 케리어를 펼치기 불편합니다.,케리어를 펼치기에 공간이 충분했습니다.,contradiction,0.000112,0.999817,0.000071,0.999817
1,1,그리고 위치가 시먼역보다는 샤오난먼역에 가까워요,시먼역보다는 샤오난먼역에 먼저 도착할 수 있어요,neutral,0.003301,0.000062,0.996636,0.996636
2,2,구구절절 설명하고 이해시키려는 노력이 큰 의미없이 다가온다.,무엇인가 말을 많이 하기는 했지만 큰 의미가 있지는 않았다.,entailment,0.999655,0.000170,0.000176,0.999655
3,3,몇 번을 다시봐도 볼 때마다 가슴이 저민다.,다시 봤을때는 무덤덤했다.,contradiction,0.000111,0.999809,0.000080,0.999809
4,4,"8월 중에 입주신청을 하면 청년은 9월, 신혼부부는 10월부터 입주가 가능하다.",8월 중에 입주신청을 하면 신혼부부는 9월 부터 입주가 가능하다.,contradiction,0.000479,0.998675,0.000845,0.998675
...,...,...,...,...,...,...,...,...
1661,1661,"또 작업자의 숙련도와 경험 향상, 전문성을 요구하는 난이도 높은 데이터 가공을 통해...",이미지 데이터를 가공하는 것이 가장 난이도가 높다.,neutral,0.000133,0.000074,0.999793,0.999793
1662,1662,결말을 보니 아무래도 이 영화는 2부가 계획된 듯 합니다.,결말을 보니 분명히 2편이 나올것이 틀림없다.,entailment,0.904698,0.000615,0.094687,0.904698
1663,1663,사회적 거리 두기 상황에서 총리도 카페를 갔다가 자리가 없어서 퇴짜 맞은 일도 있을...,사회적 거리두기 상황에서는 고위관직자도 방역지침을 준수해야 한다.,neutral,0.000555,0.000064,0.999381,0.999381
1664,1664,로마에서 3박4일간 이곳에서 머물렀습니다.,이곳에서 머무르며 로마의 명소들을 방문했습니다.,neutral,0.000270,0.000054,0.999676,0.999676


In [417]:
for i, sent in enumerate(test['premise'].unique()):
  labels = [0,1,2]
  if len(test[test['premise'] == sent]) == 3:
    test.loc[(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[0]['index']), 'label'] = test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[0][labels].index[np.argmax(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[0][labels])]
    labels.remove(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[0][labels].index[np.argmax(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[0][labels])])
    test.loc[(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[1]['index']), 'label'] = test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[1][labels].index[np.argmax(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[1][labels])]
    labels.remove(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[1][labels].index[np.argmax(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[1][labels])])
    test.loc[(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[2]['index']), 'label'] = labels[0]
  else :
    test.loc[(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[0]['index']), 'label'] = np.argmax(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[0][labels])

In [419]:
test['label'] = pd.DataFrame(num_to_label(test['label']))[1]
test

,index,premise,hypothesis,label,0,1,2,max
0,0,다만 조금 좁아서 케리어를 펼치기 불편합니다.,케리어를 펼치기에 공간이 충분했습니다.,contradiction,0.000112,0.999817,0.000071,0.999817
1,1,그리고 위치가 시먼역보다는 샤오난먼역에 가까워요,시먼역보다는 샤오난먼역에 먼저 도착할 수 있어요,entailment,0.003301,0.000062,0.996636,0.996636
2,2,구구절절 설명하고 이해시키려는 노력이 큰 의미없이 다가온다.,무엇인가 말을 많이 하기는 했지만 큰 의미가 있지는 않았다.,entailment,0.999655,0.000170,0.000176,0.999655
3,3,몇 번을 다시봐도 볼 때마다 가슴이 저민다.,다시 봤을때는 무덤덤했다.,contradiction,0.000111,0.999809,0.000080,0.999809
4,4,"8월 중에 입주신청을 하면 청년은 9월, 신혼부부는 10월부터 입주가 가능하다.",8월 중에 입주신청을 하면 신혼부부는 9월 부터 입주가 가능하다.,contradiction,0.000479,0.998675,0.000845,0.998675
...,...,...,...,...,...,...,...,...
1661,1661,"또 작업자의 숙련도와 경험 향상, 전문성을 요구하는 난이도 높은 데이터 가공을 통해...",이미지 데이터를 가공하는 것이 가장 난이도가 높다.,neutral,0.000133,0.000074,0.999793,0.999793
1662,1662,결말을 보니 아무래도 이 영화는 2부가 계획된 듯 합니다.,결말을 보니 분명히 2편이 나올것이 틀림없다.,entailment,0.904698,0.000615,0.094687,0.904698
1663,1663,사회적 거리 두기 상황에서 총리도 카페를 갔다가 자리가 없어서 퇴짜 맞은 일도 있을...,사회적 거리두기 상황에서는 고위관직자도 방역지침을 준수해야 한다.,neutral,0.000555,0.000064,0.999381,0.999381
1664,1664,로마에서 3박4일간 이곳에서 머물렀습니다.,이곳에서 머무르며 로마의 명소들을 방문했습니다.,neutral,0.000270,0.000054,0.999676,0.999676


In [420]:
test['label'].value_counts()

contradiction    556
entailment       555
neutral          555
Name: label, dtype: int64

In [421]:
test

,index,premise,hypothesis,label,0,1,2,max
0,0,다만 조금 좁아서 케리어를 펼치기 불편합니다.,케리어를 펼치기에 공간이 충분했습니다.,contradiction,0.000112,0.999817,0.000071,0.999817
1,1,그리고 위치가 시먼역보다는 샤오난먼역에 가까워요,시먼역보다는 샤오난먼역에 먼저 도착할 수 있어요,entailment,0.003301,0.000062,0.996636,0.996636
2,2,구구절절 설명하고 이해시키려는 노력이 큰 의미없이 다가온다.,무엇인가 말을 많이 하기는 했지만 큰 의미가 있지는 않았다.,entailment,0.999655,0.000170,0.000176,0.999655
3,3,몇 번을 다시봐도 볼 때마다 가슴이 저민다.,다시 봤을때는 무덤덤했다.,contradiction,0.000111,0.999809,0.000080,0.999809
4,4,"8월 중에 입주신청을 하면 청년은 9월, 신혼부부는 10월부터 입주가 가능하다.",8월 중에 입주신청을 하면 신혼부부는 9월 부터 입주가 가능하다.,contradiction,0.000479,0.998675,0.000845,0.998675
...,...,...,...,...,...,...,...,...
1661,1661,"또 작업자의 숙련도와 경험 향상, 전문성을 요구하는 난이도 높은 데이터 가공을 통해...",이미지 데이터를 가공하는 것이 가장 난이도가 높다.,neutral,0.000133,0.000074,0.999793,0.999793
1662,1662,결말을 보니 아무래도 이 영화는 2부가 계획된 듯 합니다.,결말을 보니 분명히 2편이 나올것이 틀림없다.,entailment,0.904698,0.000615,0.094687,0.904698
1663,1663,사회적 거리 두기 상황에서 총리도 카페를 갔다가 자리가 없어서 퇴짜 맞은 일도 있을...,사회적 거리두기 상황에서는 고위관직자도 방역지침을 준수해야 한다.,neutral,0.000555,0.000064,0.999381,0.999381
1664,1664,로마에서 3박4일간 이곳에서 머물렀습니다.,이곳에서 머무르며 로마의 명소들을 방문했습니다.,neutral,0.000270,0.000054,0.999676,0.999676


In [390]:
submission = pd.read_csv('/content/drive/MyDrive/KLUE/sample_submission.csv')

In [391]:
submission['label'] = test['label']

In [392]:
submission

,index,label
0,0,contradiction
1,1,entailment
2,2,entailment
3,3,contradiction
4,4,contradiction
...,...,...
1661,1661,neutral
1662,1662,entailment
1663,1663,neutral
1664,1664,neutral


In [393]:
submission['label'].value_counts()

contradiction    556
entailment       555
neutral          555
Name: label, dtype: int64

In [394]:
submission.to_csv('/content/drive/MyDrive/sm_220228/submission_soft.csv', index=False)

## hardvoting

In [300]:
pred0 = pd.read_csv('/content/drive/MyDrive/sm_220228/pred_label0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/sm_220228/pred_label1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/sm_220228/pred_label2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/sm_220228/pred_label3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/sm_220228/pred_label4.csv')

In [301]:
test['label0'] = pred0['label']
test['label1'] = pred1['label']
test['label2'] = pred2['label']
test['label3'] = pred3['label']
test['label4'] = pred4['label']

In [302]:
df = test
df

,index,premise,hypothesis,label,0,1,2,label0,label1,label2,label3,label4
0,0,다만 조금 좁아서 케리어를 펼치기 불편합니다.,케리어를 펼치기에 공간이 충분했습니다.,contradiction,0.000112,0.999817,0.000071,contradiction,contradiction,contradiction,contradiction,contradiction
1,1,그리고 위치가 시먼역보다는 샤오난먼역에 가까워요,시먼역보다는 샤오난먼역에 먼저 도착할 수 있어요,neutral,0.003301,0.000062,0.996636,neutral,neutral,neutral,neutral,neutral
2,2,구구절절 설명하고 이해시키려는 노력이 큰 의미없이 다가온다.,무엇인가 말을 많이 하기는 했지만 큰 의미가 있지는 않았다.,entailment,0.999655,0.000170,0.000176,entailment,entailment,entailment,entailment,entailment
3,3,몇 번을 다시봐도 볼 때마다 가슴이 저민다.,다시 봤을때는 무덤덤했다.,contradiction,0.000111,0.999809,0.000080,contradiction,contradiction,contradiction,contradiction,contradiction
4,4,"8월 중에 입주신청을 하면 청년은 9월, 신혼부부는 10월부터 입주가 가능하다.",8월 중에 입주신청을 하면 신혼부부는 9월 부터 입주가 가능하다.,contradiction,0.000479,0.998675,0.000845,contradiction,contradiction,contradiction,contradiction,contradiction
...,...,...,...,...,...,...,...,...,...,...,...,...
1661,1661,"또 작업자의 숙련도와 경험 향상, 전문성을 요구하는 난이도 높은 데이터 가공을 통해...",이미지 데이터를 가공하는 것이 가장 난이도가 높다.,neutral,0.000133,0.000074,0.999793,neutral,neutral,neutral,neutral,neutral
1662,1662,결말을 보니 아무래도 이 영화는 2부가 계획된 듯 합니다.,결말을 보니 분명히 2편이 나올것이 틀림없다.,entailment,0.904698,0.000615,0.094687,entailment,entailment,entailment,entailment,entailment
1663,1663,사회적 거리 두기 상황에서 총리도 카페를 갔다가 자리가 없어서 퇴짜 맞은 일도 있을...,사회적 거리두기 상황에서는 고위관직자도 방역지침을 준수해야 한다.,neutral,0.000555,0.000064,0.999381,neutral,neutral,neutral,neutral,neutral
1664,1664,로마에서 3박4일간 이곳에서 머물렀습니다.,이곳에서 머무르며 로마의 명소들을 방문했습니다.,neutral,0.000270,0.000054,0.999676,neutral,neutral,neutral,neutral,neutral


In [303]:
def label_count(df):
  num_neutral = 0
  num_contradiction = 0
  num_entailment = 0
  cols = ['label0', 'label1', 'label2', 'label3', 'label4']

  for col in cols:
    if df[col] == 'neutral':
      num_neutral = num_neutral + 1
    if df[col] == 'contradiction':
      num_contradiction = num_contradiction + 1
    if df[col] == 'entailment':
      num_entailment = num_entailment + 1

  return [num_neutral, num_contradiction, num_entailment]

temp = df.apply(label_count, axis=1)
df['temp'] = temp

df['neutral'] = 0
df['contradiction'] = 0
df['entailment'] = 0

def list_to_num(list:list):
  return list[0]
df['neutral'] = df['temp'].map(list_to_num)

def list_to_num(list:list):
  return list[1]
df['contradiction'] = df['temp'].map(list_to_num)

def list_to_num(list:list):
  return list[2]
df['entailment'] = df['temp'].map(list_to_num)

def voting(df):
  cols = ['neutral', 'contradiction', 'entailment']
  for col in cols:
    if df[col] > 2:
      return col
  return 'neutral'

df['label'] = df.apply(voting, axis=1)

In [304]:
df[(df['neutral'] != 5) & (df['contradiction'] != 5) & (df['entailment'] != 5)]

,index,premise,hypothesis,label,0,1,2,label0,label1,label2,label3,label4,temp,neutral,contradiction,entailment
6,6,18일 귀국이라 발인도 지켜드리지 못해 더욱 죄송할 따름입니다.,18일 배를 타고 여행을 떠났습니다.,neutral,0.000276,0.459825,0.539899,neutral,contradiction,contradiction,neutral,neutral,"[3, 2, 0]",3,2,0
7,7,2003년 변정수 씨가 교통사고로 죽었다는 이야기가 돌았지만 사실무근이었다.,2003년 변정수 씨는 교통사고를 당했다.,contradiction,0.008633,0.510456,0.480910,neutral,neutral,contradiction,contradiction,contradiction,"[2, 3, 0]",2,3,0
9,9,2015년 10월 임관해 사건 당시 대위였던 손씨는 수개월 뒤인 2018년 7월에 ...,손씨가 전역한 이후에 사건이 일어났다.,neutral,0.354291,0.388513,0.257196,entailment,entailment,contradiction,contradiction,neutral,"[1, 2, 2]",1,2,2
16,16,마지막 아이가 우는 장면과 최민수가 맞는 장면이 교차되는게 왜 이리 슬프던지,마지막 장면은 아주 슬펐다.,neutral,0.012451,0.377746,0.609804,neutral,contradiction,neutral,contradiction,neutral,"[3, 2, 0]",3,2,0
20,20,경제를 같이 잘 살려보자는 취지로 시작된 정책이니만큼 일본당국도 빠르게 잘 지급하도...,일본당국은 지급할 노력을 하지 않을 것으로 예상된다.,contradiction,0.000097,0.642568,0.357335,contradiction,neutral,neutral,contradiction,contradiction,"[2, 3, 0]",2,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629,1629,그러나 인양하지 않고 사고 해역을 추념공간으로 정해 인양비용을 절감하겠다는 일각의 ...,결과적으로 인양을 하지 않고 사고 해역을 추념공간으로 정했다.,neutral,0.167382,0.123585,0.709033,contradiction,neutral,entailment,neutral,neutral,"[3, 1, 1]",3,1,1
1638,1638,경기에서 대한민국은 구자철과 지동원의 활약에 힘입어 주도권을 잡았으나 이후 게인리흐...,경기에서 게인리흐에게 두 골을 내준 이후 대한민국은 주도권을 잡지 못했다.,contradiction,0.246776,0.696764,0.056461,contradiction,entailment,contradiction,contradiction,entailment,"[0, 3, 2]",0,3,2
1644,1644,10명이 함께 사용하기 불편함없이 만족했다.,함께 사용하기에 10명은 너무 많은 숫자이다.,contradiction,0.000045,0.799890,0.200065,contradiction,contradiction,contradiction,contradiction,neutral,"[1, 4, 0]",1,4,0
1656,1656,"1934년 7월, 사이토 내각을 이어 출범한 오카다 게이스케 내각은 과격파 청년 장...",오카다 게이스케 내각은 점령지 주민들의 갈등을 해결했다.,neutral,0.000083,0.104353,0.895563,neutral,neutral,neutral,contradiction,neutral,"[4, 1, 0]",4,1,0


In [305]:
def make_label(list:list):
  if list == [3, 2, 0] :
    return 'contradiction'
  if list == [3, 0, 2]:
    return 'entailment'
  if list == [4, 1, 0]:
    return 'contradiction'
  if list == [4, 0, 1]:
    return 'neutral'
  if list == [3, 1, 1]:
    return 'contradiction'
  if list == [1, 2, 2]:
    return 'contradiction'
  else :
    return 'answer'

my_label = df['temp'].map(make_label)

In [306]:
df['my_label'] = my_label

In [307]:
my_answer = df[df['my_label'] != 'answer']

In [308]:
df.loc[my_answer.index, 'label'] = my_answer['my_label']

In [309]:
df['label'].value_counts()

contradiction    559
neutral          558
entailment       549
Name: label, dtype: int64

In [315]:
df[(df['label'] == 'neutral') & (df['entailment'] != 0)]['premise'].loc[42]

'비과학적인 것은 일체 믿지 않는 잡지 편집자 진초게 하나메는 담당하는 잡지가 휴간되자 회사를 그만둔다.'

In [316]:
df[(df['label'] == 'neutral') & (df['entailment'] != 0)]

,index,premise,hypothesis,label,0,1,2,label0,label1,label2,label3,label4,temp,neutral,contradiction,entailment,my_label
42,42,비과학적인 것은 일체 믿지 않는 잡지 편집자 진초게 하나메는 담당하는 잡지가 휴간되...,진초게는 회사를 옮기게 된다.,neutral,0.191415,0.000149,0.808436,neutral,entailment,neutral,neutral,neutral,"[4, 0, 1]",4,0,1,neutral
124,124,서로 다른 삶 속에서 서로 같은 것을 이루며 살아가는 사람들,사람들은 같은 목표를 가지고 다른 환경에서 살아간다.,neutral,0.188064,0.010121,0.801815,neutral,neutral,neutral,entailment,neutral,"[4, 0, 1]",4,0,1,neutral
267,267,"바닷가의 고운모래와 맑은 바닷물, 잔잔한 파도 등은 해수욕하기에도 적합하다.",바닷가가 수영하기에 적합하다.,neutral,0.220961,0.000281,0.778758,neutral,neutral,entailment,neutral,neutral,"[4, 0, 1]",4,0,1,neutral
282,282,모건 프리먼과 케빈 스페이시가 아깝다.,케빈 스페이시는 낭비되기에 아쉽다.,neutral,0.267831,0.023855,0.708315,neutral,neutral,neutral,entailment,neutral,"[4, 0, 1]",4,0,1,neutral
318,318,"경찰 조사 결과, 박군은 전날 오후부터 친구들과 함께 렌터카를 타고 다니다 사고를 ...",박군은 친구들과 렌터카를 타고 다니다 사고가 나자 경찰의 조사를 받았다.,neutral,0.224157,0.000082,0.775762,neutral,neutral,entailment,neutral,neutral,"[4, 0, 1]",4,0,1,neutral
436,436,따라서 도심 집회 관련 감염에 대한 수치를 포함하지 않은 해당 기사는 근거가 불충분하다.,해당 기사는 도심 집회 관련 감염에 대한 수치를 정확하게 포함하고 있지 않다.,neutral,0.203967,0.000072,0.795961,neutral,neutral,neutral,neutral,entailment,"[4, 0, 1]",4,0,1,neutral
463,463,"해상의 경우는 26일 일요일 밤의 제주도 남쪽 먼바다를 시작으로, 27일 월요일에 ...",해상에 태풍 특보는 27일 월요일 부터 확대 되었다.,neutral,0.154054,0.001388,0.844558,neutral,entailment,neutral,neutral,neutral,"[4, 0, 1]",4,0,1,neutral
774,774,샤워기가 위에만 있다는게 아쉬운 단점이였어요.,세면대와 샤워기가 아쉬웠어요.,neutral,0.284785,0.000329,0.714886,neutral,neutral,neutral,entailment,neutral,"[4, 0, 1]",4,0,1,neutral
1128,1128,서울특별시가 11월 13일 부터 광화문 광장에 위치한 이순신 장군 동상에 대한 정밀...,이순신 장군 동상 정밀 보수 공사를 시작한지 13일이 지났다.,neutral,0.143279,0.061915,0.794806,neutral,neutral,neutral,neutral,entailment,"[4, 0, 1]",4,0,1,neutral
1192,1192,1주택을 소유한 1세대가 다른 1주택을 추가로 취득한 경우 종전 주택을 일정 기간 ...,1주택을 소유한 1세대가 다른 1주택을 추가로 취득한 경우 거주하던 주택을 일정 기...,neutral,0.124374,0.459609,0.416017,neutral,contradiction,contradiction,neutral,entailment,"[2, 2, 1]",2,2,1,answer


In [ ]:
# neutral          562
# contradiction    555
# entailment       549

In [310]:
submission = pd.read_csv('/content/drive/MyDrive/KLUE/sample_submission.csv')

In [311]:
submission['label'] = df['label']

In [312]:
submission

,index,label
0,0,contradiction
1,1,neutral
2,2,entailment
3,3,contradiction
4,4,contradiction
...,...,...
1661,1661,neutral
1662,1662,entailment
1663,1663,neutral
1664,1664,neutral


In [313]:
submission.to_csv('/content/drive/MyDrive/sm_220228/submission_hard.csv', index=False)

## soft + hard voting

In [270]:
soft = pd.read_csv('/content/drive/MyDrive/sm_220228/submission_soft.csv')
hard = pd.read_csv('/content/drive/MyDrive/sm_220228/submission_hard.csv')

In [268]:
df

,index,premise,hypothesis,label,0,1,2,label0,label1,label2,label3,label4,temp,neutral,contradiction,entailment,my_label
0,0,다만 조금 좁아서 케리어를 펼치기 불편합니다.,케리어를 펼치기에 공간이 충분했습니다.,contradiction,0.000112,0.999817,0.000071,contradiction,contradiction,contradiction,contradiction,contradiction,"[0, 5, 0]",0,5,0,answer
1,1,그리고 위치가 시먼역보다는 샤오난먼역에 가까워요,시먼역보다는 샤오난먼역에 먼저 도착할 수 있어요,neutral,0.003301,0.000062,0.996636,neutral,neutral,neutral,neutral,neutral,"[5, 0, 0]",5,0,0,answer
2,2,구구절절 설명하고 이해시키려는 노력이 큰 의미없이 다가온다.,무엇인가 말을 많이 하기는 했지만 큰 의미가 있지는 않았다.,entailment,0.999655,0.000170,0.000176,entailment,entailment,entailment,entailment,entailment,"[0, 0, 5]",0,0,5,answer
3,3,몇 번을 다시봐도 볼 때마다 가슴이 저민다.,다시 봤을때는 무덤덤했다.,contradiction,0.000111,0.999809,0.000080,contradiction,contradiction,contradiction,contradiction,contradiction,"[0, 5, 0]",0,5,0,answer
4,4,"8월 중에 입주신청을 하면 청년은 9월, 신혼부부는 10월부터 입주가 가능하다.",8월 중에 입주신청을 하면 신혼부부는 9월 부터 입주가 가능하다.,contradiction,0.000479,0.998675,0.000845,contradiction,contradiction,contradiction,contradiction,contradiction,"[0, 5, 0]",0,5,0,answer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1661,1661,"또 작업자의 숙련도와 경험 향상, 전문성을 요구하는 난이도 높은 데이터 가공을 통해...",이미지 데이터를 가공하는 것이 가장 난이도가 높다.,neutral,0.000133,0.000074,0.999793,neutral,neutral,neutral,neutral,neutral,"[5, 0, 0]",5,0,0,answer
1662,1662,결말을 보니 아무래도 이 영화는 2부가 계획된 듯 합니다.,결말을 보니 분명히 2편이 나올것이 틀림없다.,entailment,0.904698,0.000615,0.094687,entailment,entailment,entailment,entailment,entailment,"[0, 0, 5]",0,0,5,answer
1663,1663,사회적 거리 두기 상황에서 총리도 카페를 갔다가 자리가 없어서 퇴짜 맞은 일도 있을...,사회적 거리두기 상황에서는 고위관직자도 방역지침을 준수해야 한다.,neutral,0.000555,0.000064,0.999381,neutral,neutral,neutral,neutral,neutral,"[5, 0, 0]",5,0,0,answer
1664,1664,로마에서 3박4일간 이곳에서 머물렀습니다.,이곳에서 머무르며 로마의 명소들을 방문했습니다.,neutral,0.000270,0.000054,0.999676,neutral,neutral,neutral,neutral,neutral,"[5, 0, 0]",5,0,0,answer


In [271]:
df = df[['premise',	'hypothesis',	'label',	0,	1,	2,	'label0',	'label1',	'label2',	'label3',	'label4']]

In [272]:
df['soft'] = soft['label']
df['hard'] = hard['label']

In [273]:
df['soft'].value_counts()

neutral          605
entailment       540
contradiction    521
Name: soft, dtype: int64

In [274]:
df['hard'].value_counts()

contradiction    559
neutral          558
entailment       549
Name: hard, dtype: int64

In [201]:
df.loc[df[(df['hard'] != df['soft']) & (df['soft'] == 'neutral')].index, 'label'] = df[(df['hard'] != df['soft']) & (df['soft'] == 'neutral')]['soft']

In [202]:
df.loc[df[(df['hard'] != df['soft']) & (df['soft'] != 'neutral')].index, 'label'] = df[(df['hard'] != df['soft']) & (df['soft'] != 'neutral')]['hard']

In [203]:
df['label'].value_counts()

neutral          610
entailment       537
contradiction    519
Name: label, dtype: int64

In [204]:
submission = pd.read_csv('/content/drive/MyDrive/KLUE/sample_submission.csv')

In [205]:
submission['label'] = df['label']

In [206]:
submission

,index,label
0,0,contradiction
1,1,neutral
2,2,entailment
3,3,contradiction
4,4,contradiction
...,...,...
1661,1661,neutral
1662,1662,entailment
1663,1663,neutral
1664,1664,neutral


In [207]:
submission.to_csv('/content/drive/MyDrive/sm_220228/submission_soft_hard.csv', index=False)